In [2]:
import pandas as pd
data = pd.DataFrame(columns = ['S No.','Reco Full','Reco Datetime','Reco Provider','Reco For','Symbol','Target Price','Action','Buy Date','Buy Price','Sell Date','Sell Price','Status'])
data

,S No.,Reco Full,Reco Datetime,Reco Provider,Reco For,Symbol,Target Price,Action,Buy Date,Buy Price,Sell Date,Sell Price,Status


In [3]:
def convert_to_float(str):
    '''Converts pricing to a float representation'''
    floattext = 0.0 
    if "," in str:
        str = str.replace(",","")
        floattext = float(str)
    else:
        floattext = float(str)
    return floattext

In [4]:
buy = ['buy','accumulate','hold']
sell = ['sell','reduce']

In [5]:
import datetime
import threading
import os
import shutil

today = datetime.date.today()

text = today.strftime('%d%m%y')
print(text)
file = None
def getData():
    global file
    global text
    file= requests.get(f'https://archives.nseindia.com/archives/equities/bhavcopy/pr/PR{text}.zip',allow_redirects=True)
thread = threading.Thread(target=getData)
from zipfile import ZipFile
import requests
while True:
    file = None
    thread.start()
    thread.join(timeout=1.5)
    # print("Thread Ended")
    if file == None:
        today = today - datetime.timedelta(1)
        text = today.strftime('%d%m%y')
        thread = threading.Thread(target=getData)
        # print(text)
        continue
    else:
        break
open('test.zip', 'wb').write(file.content)
try:    
    shutil.rmtree('BhavCopy')
except FileNotFoundError:
    pass
os.mkdir('BhavCopy')
with ZipFile('test.zip', 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    zip.extract(f'Pd{text}.csv',path='BhavCopy')
    

160422


In [6]:
bhavcopy = pd.read_csv(f'BhavCopy/Pd{text}.csv')
terminate = len(bhavcopy['SECURITY'])
for x in range(len(bhavcopy['SECURITY'])):
    if bhavcopy['SECURITY'][x] == 'OTHER SECURITIES':
        terminate = x-1
name = pd.DataFrame()
name['SECURITY'] = bhavcopy['SECURITY']
name['SYMBOL'] = bhavcopy['SYMBOL']
name = name[0:terminate]
name

,SECURITY,SYMBOL
0,Nifty 50,
1,NIFTY50 EQL Wgt,
2,NIFTY100 EQL Wgt,
3,NIFTY100 LowVol30,
4,NIFTY Alpha 50,
...,...,...
1947,UNITECH LIMITED,UNITECH
1948,VICEROY HOTELS LIMITED,VICEROY
1949,EMPYREAN CASHEWS LIMITED,KRISHIVAL
1950,KRISHNA DEF AND ALD IND L,KRISHNADEF


In [7]:
name[0:11]

,SECURITY,SYMBOL
0,Nifty 50,
1,NIFTY50 EQL Wgt,
2,NIFTY100 EQL Wgt,
3,NIFTY100 LowVol30,
4,NIFTY Alpha 50,
5,NIFTY MIDCAP 150,
6,NIFTY SMLCAP 50,
7,NIFTY SMLCAP 250,
8,NIFTY MIDSML 400,
9,NIFTY200 QUALTY30,


In [8]:
import json
def symbol_converter(to_sein):
    to_search = to_sein
    file=open('similar_comp.json')
    similar_words = json.load(file)
    file.close()
    allWords = []
    for x in similar_words.values():
        for z in x:
            allWords.append(z)
    # Removes any special characters
    for x in to_search:
        if not x.isalnum():
            to_search = to_search.replace(x," ")
    to_search = to_search.upper()
    
    match = []
    for i in name['SECURITY']:
        temp = 0
        if i in allWords:
            for x in similar_words.values():
                if i in x:
                    for z in x:
                        if z == to_search:
                            temp = 1
                            break
                        else:
                            continue
        elif len(to_search.split())==1 and (to_search in i.split()):
            temp = 1
        elif len(to_search.split()) > 1:
            bool = None
            if to_search.split()[:1] == i.split()[:1]:
                temp += 0.1
            for x in to_search.split():
                if x in i:
                    temp += 1/len(to_search.split())
                else:
                    for z in similar_words.values():
                        if x in z:
                            for word in z:
                                if word in i:
                                    temp += 1/len(to_search.split())
        else:
            temp = 0   
        match.append(temp)

    name['Match'] = match

    sorted_name = name.sort_values(by = ['Match'], ascending =False)
    if sorted_name.iloc[0,0] == 'Nifty 50' and to_search in name['SYMBOL'].to_list():
        return to_search
    elif sorted_name.iloc[0,0] == 'Nifty 50':
        return "N/A"
    else:
        return sorted_name.iloc[0,1]


In [9]:
"Sobha ".strip()

'Sobha'

In [10]:
def trader(recodate,stock,target,action):
    if datetime.time(15,30,00) > recodate.time():
        stockdata = nsepy.get_history(stock,recodate.date(),datetime.datetime.date(datetime.datetime.now()))
    else:
        stockdata = nsepy.get_history(stock,(recodate.date() + datetime.timedelta(days=1)),datetime.datetime.date(datetime.datetime.now()))
    
    if len(stockdata) == 0:
        return ['N/A','N/A','N/A','N/A','Not Attempted']
    elif target < stockdata.iloc[0,3]/2.5 or target > stockdata.iloc[0,3]*2.5:
        return ['N/A','N/A','N/A','N/A','Not Attempted']
    elif action and stockdata.iloc[0,3]<target:
        for x in range(len(list(stockdata['High']))):
            # print(list(stockdata['High'])[x])
            if list(stockdata['Low'])[x] <= (0.9*stockdata.iloc[0,3]):
                return [stockdata.iloc[0,3],list(stockdata.index)[0],0.9*stockdata.iloc[0,3],list(stockdata.index)[x],'Complete']
            elif list(stockdata['High'])[x] >= target:
                # print(10)
                return [stockdata.iloc[0,3],list(stockdata.index)[0],target,list(stockdata.index)[x],'Complete']
            elif x == len(list(stockdata['High']))-1 :
                # print(100)
                return [stockdata.iloc[0,3],list(stockdata.index)[0],'N/A','N/A','Open']
    elif action == False and stockdata.iloc[0,3]>target:
        for x in range(len(list(stockdata['Low']))):
            if list(stockdata['High'])[x] >= (1.1*stockdata.iloc[0,3]):
                return [1.1*stockdata.iloc[0,3],list(stockdata.index)[x],stockdata.iloc[0,3],list(stockdata.index)[0],'Complete']
            elif list(stockdata['Low'])[x] <= target:
                return [target,list(stockdata.index)[x],stockdata.iloc[0,3],list(stockdata.index)[0],'Complete']
            elif x == len(list(stockdata['Low']))-1 :
                return ['N/A','N/A',stockdata.iloc[0,3],list(stockdata.index)[0],'Open']
    else:
        return ['N/A','N/A','N/A','N/A','Not Attempted']           


In [11]:
data = pd.read_csv('Broker Trades2.csv')
data

,S No.,Reco Full,Reco Datetime,Reco Provider,Reco For,Symbol,Target Price,Action,Buy Date,Buy Price,Sell Date,Sell Price,Status
0,1,Buy Tata Consultancy Services; target of Rs 43...,2022-04-14 20:43:00,Anand Rathi,Tata Consultancy Services,TCS,4350.0,Buy,NaN,NaN,NaN,NaN,Not Attempted
1,2,Accumulate Infosys; target of Rs 1899: Prabhud...,2022-04-14 13:42:00,Prabhudas Lilladher,Infosys,INFY,1899.0,Accumulate,NaN,NaN,NaN,NaN,Not Attempted
2,3,Buy Infosys; target of Rs 1970: Emkay Global F...,2022-04-14 11:05:00,Emkay Global Financial,Infosys,INFY,1970.0,Buy,NaN,NaN,NaN,NaN,Not Attempted
3,4,Buy Infosys; target of Rs 2000: Motilal Oswal,2022-04-14 09:03:00,Motilal Oswal,Infosys,INFY,2000.0,Buy,NaN,NaN,NaN,NaN,Not Attempted
4,5,Buy Infosys: target of Rs 2150: Sharekhan,2022-04-14 08:35:00,Sharekhan,Infosys,INFY,2150.0,Buy,NaN,NaN,NaN,NaN,Not Attempted
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088,6089,Buy Coal India; target of Rs 338: Motilal Oswal,2019-02-13 15:35:00,Motilal Oswal,Coal India,COALINDIA,338.0,Buy,2019-02-14,217.00,2019-08-05,195.300,Complete
6089,6090,Buy Allcargo Logistics; target of Rs 142: Moti...,2019-02-13 15:33:00,Motilal Oswal,Allcargo Logistics,ALLCARGO,142.0,Buy,2019-02-14,105.70,2019-07-30,95.130,Complete
6090,6091,Buy PI Industries; target of Rs 1023: Motilal ...,2019-02-13 15:28:00,Motilal Oswal,PI Industries,PIIND,1023.0,Buy,2019-02-13,880.50,2019-03-12,1023.000,Complete
6091,6092,Buy Indian Hotels Co; target of Rs 189: Motila...,2019-02-13 15:25:00,Motilal Oswal,Indian Hotels Co,INDHOTEL,189.0,Buy,2019-02-13,141.95,2019-08-21,127.755,Complete


In [12]:
import nsepy
titles = []
dates = []
keys = ['S No.','Reco Full','Reco Datetime','Reco Provider','Reco For','Symbol','Target Price','Action','Buy Date','Buy Price','Sell Date','Sell Price','Status']
# ['S No.','Reco Full','Reco Datetime','Reco Provider','Reco For','Symbol','Target Price','Action',)

dictdata = dict.fromkeys(keys)
import requests
import math
from bs4 import BeautifulSoup
for x in range(1,1000):
    try:
        webpage = requests.get(f'https://www.moneycontrol.com/broker-research/markets/cash-{x}.html')
    except:
        continue
    soup = BeautifulSoup(webpage.text, 'html.parser')
    for i in range(1):
        try:
            title = soup.find_all('section',class_="colLft_in_broker")[i].get_text()
        
            # title = soup.li
            # date = soup.find_all('li')[i].get_text()
            
            title = title.split('\n')
            # print(title)
            for x in range(len(title)):
                if 'target' in title[x] and title[x] not in list(data['Reco Full']):
                    # print(x)
                    dictdata['Reco Full'] = title[x]
                    dictdata['Reco Datetime'] = datetime.datetime.strptime(' '.join(title[x-1].split(" | ")[1:])," %b %d, %Y %I:%M %p")
                    dictdata['Action']= dictdata['Reco Full'].split(" ")[0]
                    for f in range(len(dictdata['Reco Full'].split(" "))):
                        if ':' in (dictdata['Reco Full'].split(" "))[f]:
                            dictdata['Reco For'] = ((" ".join(dictdata['Reco Full'].split(" ")[1:f+1])).replace(":","")).strip()
                        elif ';' in (dictdata['Reco Full'].split(" "))[f]:
                            dictdata['Reco For'] = ((" ".join(dictdata['Reco Full'].split(" ")[1:f+1])).replace(";","")).strip()
                        elif 'Rs' in (dictdata['Reco Full'].split(" "))[f]:
                            try:
                                dictdata['Target Price'] = convert_to_float(("".join(dictdata['Reco Full'].split(" ")[f+1:f+2])).replace(":",""))
                            except:
                                continue
                            dictdata['Reco Provider'] = (" ".join(dictdata['Reco Full'].split(" ")[f+2:len(dictdata['Reco Full'].split(" "))-1])).replace(": ","")
                            break
                    dictdata['Symbol'] = symbol_converter(dictdata['Reco For'])
                    if math.isnan(data['S No.'].max()):
                        dictdata['S No.'] = 1
                    else:
                        dictdata['S No.'] = data['S No.'].max() + 1
                    print(dictdata)
                    if dictdata['Action'].lower() in buy and dictdata['Symbol']!= 'N/A':
                        list_temp = trader(dictdata['Reco Datetime'],dictdata['Symbol'],dictdata['Target Price'],True)
                    elif dictdata['Action'].lower() in sell and dictdata['Symbol']!= 'N/A':
                        list_temp = trader(dictdata['Reco Datetime'],dictdata['Symbol'],dictdata['Target Price'],False)
                    else:
                        dictdata['Buy Price'] = 'N/A'
                        dictdata['Buy Date'] = 'N/A'
                        dictdata['Sell Price'] = 'N/A'
                        dictdata['Sell Date'] = 'N/A'
                        dictdata['Status'] = 'Not Attempted'
                        continue
                    print(list_temp)
                    dictdata['Buy Price'] = list_temp[0]
                    dictdata['Buy Date'] = list_temp[1]
                    dictdata['Sell Price'] = list_temp[2]
                    dictdata['Sell Date'] = list_temp[3]
                    dictdata['Status'] = list_temp[4]
                    data = data.append(dictdata,ignore_index=True)
        except:
            continue    
        data.to_csv('Broker Trades2.csv',index=False)        
    # print(dates)
            
            # print(date)
    # for z in range(len(titles)):
    #     print(titles[z],dates[z])

{'S No.': 6094, 'Reco Full': 'Neutral Infosys; target of Rs 1659: Arihant Capital ', 'Reco Datetime': datetime.datetime(2022, 4, 16, 15, 13), 'Reco Provider': 'Arihant Capital', 'Reco For': 'Infosys', 'Symbol': 'INFY', 'Target Price': 1659.0, 'Action': 'Neutral', 'Buy Date': None, 'Buy Price': None, 'Sell Date': None, 'Sell Price': None, 'Status': None}
{'S No.': 6094, 'Reco Full': 'Neutral Infosys; target of Rs 1659: Arihant Capital ', 'Reco Datetime': datetime.datetime(2022, 4, 16, 15, 13), 'Reco Provider': 'Arihant Capital', 'Reco For': 'Infosys', 'Symbol': 'INFY', 'Target Price': 1659.0, 'Action': 'Neutral', 'Buy Date': 'N/A', 'Buy Price': 'N/A', 'Sell Date': 'N/A', 'Sell Price': 'N/A', 'Status': 'Not Attempted'}
{'S No.': 6094, 'Reco Full': 'Buy Infosys; target of Rs 2094: KR Choksey ', 'Reco Datetime': datetime.datetime(2022, 4, 16, 14, 57), 'Reco Provider': 'KR Choksey', 'Reco For': 'Infosys', 'Symbol': 'INFY', 'Target Price': 2094.0, 'Action': 'Buy', 'Buy Date': 'N/A', 'Buy Pri

Exception in thread Thread-6 (getData):
Traceback (most recent call last):
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1374, in getresponse
    response.begin(

[230.1, datetime.date(2019, 2, 8), 315.0, datetime.date(2019, 11, 14), 'Complete']
{'S No.': 6280, 'Reco Full': 'Buy Indraprastha Gas; target of Rs 368: Dolat Capital ', 'Reco Datetime': datetime.datetime(2019, 2, 8, 15, 24), 'Reco Provider': 'Dolat Capital', 'Reco For': 'Indraprastha Gas', 'Symbol': 'IGL', 'Target Price': 368.0, 'Action': 'Buy', 'Buy Date': datetime.date(2019, 2, 8), 'Buy Price': 230.1, 'Sell Date': datetime.date(2019, 11, 14), 'Sell Price': 315.0, 'Status': 'Complete'}


Exception in thread Thread-7 (getData):
Traceback (most recent call last):
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1374, in getresponse
    response.begin(

[301.0, datetime.date(2019, 2, 8), 270.90000000000003, datetime.date(2019, 2, 11), 'Complete']
{'S No.': 6281, 'Reco Full': 'Buy Tech Mahindra; target of Rs 950: Dolat Capital ', 'Reco Datetime': datetime.datetime(2019, 2, 8, 15, 22), 'Reco Provider': 'Dolat Capital', 'Reco For': 'Tech Mahindra', 'Symbol': 'TECHM', 'Target Price': 950.0, 'Action': 'Buy', 'Buy Date': datetime.date(2019, 2, 8), 'Buy Price': 301.0, 'Sell Date': datetime.date(2019, 2, 11), 'Sell Price': 270.90000000000003, 'Status': 'Complete'}


Exception in thread Thread-8 (getData):
Traceback (most recent call last):
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1374, in getresponse
    response.begin(

[800.0, datetime.date(2019, 2, 8), 720.0, datetime.date(2019, 5, 27), 'Complete']
{'S No.': 6282, 'Reco Full': 'Buy KEI Industries; target of Rs 443: Dolat Capital ', 'Reco Datetime': datetime.datetime(2019, 2, 8, 15, 19), 'Reco Provider': 'Dolat Capital', 'Reco For': 'KEI Industries', 'Symbol': 'KEI', 'Target Price': 443.0, 'Action': 'Buy', 'Buy Date': datetime.date(2019, 2, 8), 'Buy Price': 800.0, 'Sell Date': datetime.date(2019, 5, 27), 'Sell Price': 720.0, 'Status': 'Complete'}
[359.2, datetime.date(2019, 2, 8), 323.28, datetime.date(2019, 2, 18), 'Complete']
{'S No.': 6283, 'Reco Full': 'Sell Berger Paints; target of Rs 300: Dolat Capital ', 'Reco Datetime': datetime.datetime(2019, 2, 8, 15, 15), 'Reco Provider': 'Dolat Capital', 'Reco For': 'Berger Paints', 'Symbol': 'BERGEPAINT', 'Target Price': 300.0, 'Action': 'Sell', 'Buy Date': datetime.date(2019, 2, 8), 'Buy Price': 359.2, 'Sell Date': datetime.date(2019, 2, 18), 'Sell Price': 323.28, 'Status': 'Complete'}
[300.0, datetime.

Exception in thread Thread-18904 (get_history):
Traceback (most recent call last):
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of t

{'S No.': 7217, 'Reco Full': 'Accumulate Pidilite Industries; target of Rs 1175: Prabhudas Lilladher ', 'Reco Datetime': datetime.datetime(2018, 8, 29, 17, 18), 'Reco Provider': 'Prabhudas Lilladher', 'Reco For': 'Pidilite Industries', 'Symbol': 'PIDILITIND', 'Target Price': 1175.0, 'Action': 'Accumulate', 'Buy Date': datetime.date(2018, 9, 4), 'Buy Price': 668.8, 'Sell Date': datetime.date(2018, 9, 21), 'Sell Price': 601.92, 'Status': 'Complete'}
[1147.45, datetime.date(2018, 8, 30), 1175.0, datetime.date(2018, 8, 31), 'Complete']
{'S No.': 7218, 'Reco Full': 'Buy The Phoenix Mills; target of Rs 707: Kotak Securities ', 'Reco Datetime': datetime.datetime(2018, 8, 29, 17, 17), 'Reco Provider': 'Kotak Securities', 'Reco For': 'The Phoenix Mills', 'Symbol': 'PHOENIXLTD', 'Target Price': 707.0, 'Action': 'Buy', 'Buy Date': datetime.date(2018, 8, 30), 'Buy Price': 1147.45, 'Sell Date': datetime.date(2018, 8, 31), 'Sell Price': 1175.0, 'Status': 'Complete'}
[604.0, datetime.date(2018, 8, 30

Trade Taker

In [ ]:
data['Reco Datetime']

0    2022-02-18 19:47:00
1    2022-02-18 19:46:00
2    2022-02-18 19:46:00
3    2022-02-18 19:46:00
4    2022-02-18 19:46:00
             ...        
60   2022-02-15 17:59:00
61   2022-02-15 17:59:00
62   2022-02-15 17:59:00
63   2022-02-15 17:39:00
64   2022-02-15 17:39:00
Name: Reco Datetime, Length: 65, dtype: datetime64[ns]

In [ ]:
# for i in data['S No.']:
#     a = data['Symbol'].loc[data['S No.']==i]
#     # print(a[0])
#     if datetime.time(15,30,00) > datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').time():
#         stockdata = nsepy.get_history(list(data['Symbol'].loc[data['S No.']==i])[0],datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date(),datetime.datetime.date(datetime.datetime.now()))
#     else:
#         stockdata = nsepy.get_history(list(data['Symbol'].loc[data['S No.']==i])[0],(datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date()+ datetime.timedelta(days=1)),datetime.datetime.date(datetime.datetime.now()))
    
#     print(list(data['Symbol'].loc[data['S No.']==i])[0])
#     print(datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').time())
#     print(datetime.time(15,30,00) > datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').time())
#     print(datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date())
#     print(datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date() + datetime.timedelta(days=1))
#     stockdata_1 = nsepy.get_history(list(data['Symbol'].loc[data['S No.']==i])[0],datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date(),datetime.datetime.date(datetime.datetime.now()))
#     print(stockdata_1.iloc[0,3])
    

In [ ]:
print(datetime.datetime.today())

2022-04-09 20:57:38.751774


In [ ]:
data.to_clipboard()

In [ ]:
import datetime
datetime.datetime.strptime(" Mar 25, 2022 05:45 pm"," %b %d, %Y %I:%M %p")

datetime.datetime(2022, 3, 25, 17, 45)

In [ ]:
print(datetime.datetime.date(datetime.datetime.now()))
# type(datetime.datetime.strptime((str(list(data['Reco Datetime'].loc[data['S No.']==i])[0])),'%Y-%m-%d %H:%M:%S').date())

2022-04-09


In [ ]:
import nsepy
import datetime
nsepy.get_history("KALPATPOWR",datetime.date(2022,2,22),datetime.date(2022,4,14))
# print(stockdat2)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2022-02-22,KALPATPOWR,EQ,403.35,390.00,405.00,386.20,395.50,398.25,398.83,347300,1.385146e+13,16754,76153,0.2193
2022-02-23,KALPATPOWR,EQ,398.25,400.10,404.85,397.55,400.50,401.00,401.02,291201,1.167773e+13,7705,166551,0.5719
2022-02-24,KALPATPOWR,EQ,401.00,396.95,401.75,345.00,389.00,390.75,392.59,406831,1.597186e+13,25646,176296,0.4333
2022-02-25,KALPATPOWR,EQ,390.75,392.00,402.35,392.00,399.90,400.05,398.22,156664,6.238664e+12,12222,77049,0.4918
2022-02-28,KALPATPOWR,EQ,400.05,402.00,402.00,390.00,390.80,391.10,392.47,80537,3.160845e+12,3909,32880,0.4083
2022-03-02,KALPATPOWR,EQ,391.10,391.10,398.00,383.10,384.20,387.15,390.42,73581,2.872716e+12,4249,33518,0.4555
2022-03-03,KALPATPOWR,EQ,387.15,388.00,391.65,380.60,383.00,383.50,385.34,112591,4.338593e+12,4262,56844,0.5049
2022-03-04,KALPATPOWR,EQ,383.50,383.00,386.30,370.00,371.90,371.95,376.39,82740,3.114226e+12,4809,44979,0.5436
2022-03-07,KALPATPOWR,EQ,371.95,365.00,383.95,360.35,376.00,380.60,373.89,137098,5.125972e+12,7580,49279,0.3594


In [ ]:
import datetime
datetime.datetime.today()

datetime.datetime(2022, 4, 9, 14, 8, 36, 129137)